In [1]:
!pip install open3d


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.4
    Uninstalling widgetsnbextension-3.6.4:
      Successfully uninstalled widgetsnbextension-3.6.4
  Attempting uninstall: werkzeug
    Found existing installa

**Load the requirements**

In [4]:
import os
import open3d as o3d
import cv2
import numpy as np
import glob


In [3]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow

Mounted at /content/drive


**Testing a PCD File**

In [43]:
im1 = "/content/drive/MyDrive/6 DoF Pose Estimation/cereal_box_1/segments/cereal_box_1_1_1_seg1.pcd"
pcd1=o3d.io.read_point_cloud(im1)
o3d.visualization.draw_plotly([pcd1])


**Load Dataset**

In [7]:
dataset_path = '/content/drive/MyDrive/6 DoF Pose Estimation'
dataset_path_cereal_box_1 = 'uncc-dataset/cereal_box_1/'

In [23]:
def load_segmented(directory):
    segmented = []
    pcd_files = glob.glob(os.path.join(directory, '*.pcd'))
    for pcd_file in pcd_files:
        file_name = os.path.basename(pcd_file)
        item_name = file_name.split('.')
        object_name,_,_, pose_idx, view_idx,segment= item_name[0].split('_')
        dict = {}
        dict['object_name'] = object_name
        dict['pose_idx'] = pose_idx
        dict['view_idx'] = view_idx
        dict['segment_name'] = segment
        dict['pcd'] =  o3d.io.read_point_cloud(pcd_file)

        segmented.append(dict)
  
    return segmented

segmented_cb1 = load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/cereal_box_1/segments')
segmented_cb2 = load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/cereal_box_2/segments')

segmented_cc1=load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/coffee_can_2/segments')

segmented_cookies=load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/cookie_box_1/segments')

segmented_jc1=load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/juice_container_1/segments')
segmented_jc2=load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/juice_container_2/segments')

segmented_mb1=load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/milk_box_1/segments')
segmented_mb2=load_segmented('/content/drive/MyDrive/6 DoF Pose Estimation/milk_box_2/segments')

**Keypoint descriptors using ISS Keypoints**

In [45]:


def point_cloud_descriptor(pcd):
    voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.01)

    # Compute the normals of the voxel grid
    voxel_down_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    # Detect keypoints using ISSKeypoint3D
    keypoints = o3d.geometry.keypoint.compute_iss_keypoints(voxel_down_pcd,
                                                            salient_radius=0.01,
                                                            non_max_radius=0.01,
                                                            gamma_21=0.8,
                                                            gamma_32=0.8)
    
    o3d.visualization.draw_plotly([pcd1, keypoints])

    # Convert keypoints to NumPy array
    descriptors = np.asarray(keypoints.points)

    # Reshape descriptors if necessary
    descriptors = descriptors.reshape(-1, 1)  # Reshape to a 2D array

    # Print the descriptors
    print(descriptors)

    return descriptors



In [46]:
point_cloud_descriptor(pcd1)

[]


array([], shape=(0, 1), dtype=float64)

In [11]:
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

In [12]:
def extract_sift_descriptors(image):
    descriptors = []
    sift = cv2.xfeatures2d.SIFT_create()
    _, desc = sift.detectAndCompute(image, None)
    descriptors.extend(desc)
    return np.array(descriptors)

In [13]:
def perform_kmeans_clustering(descriptors, k):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(descriptors)
    return kmeans

In [14]:
def create_histogram(descriptors, kmeans):
    histogram = np.zeros(kmeans.n_clusters)
    labels = kmeans.predict(descriptors)
    for label in labels:
        histogram[label] += 1
    return histogram

In [25]:
def object_images(segmented):
    object_images = []
    for i in range(0, len(segmented)):
      object_images.append(segmented[i]['pcd'])
    return(object_images)


In [24]:
obj=object_images(segmented_cb1)
obj2=object_images(segmented_cb2)
obj3=object_images(segmented_cc1)
obj4=object_images(segmented_cookies)
obj5=object_images(segmented_jc1)
#obj6=object_images(segmented_jc2)
obj7=object_images(segmented_mb1)
obj8=object_images(segmented_mb2)



In [28]:
def x_variable(obj):
    sift_descriptors = []
    for image in obj:
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #descriptors = extract_sift_descriptors(image)
        descriptors = point_cloud_descriptor(image)

        sift_descriptors.extend(descriptors)
    sift_descriptors = np.array(sift_descriptors)
    # Kmeans Algorithm
    k = 512  # Number of clusters
    kmeans = perform_kmeans_clustering(sift_descriptors, k)

    #Create a histogram of codeword occurrences for each object image
    histograms = []
    for image in obj:
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        descriptors = extract_sift_descriptors(image)
        histogram = create_histogram(descriptors, kmeans)
        histograms.append(histogram)

    #Visualize the Histogram
    plt.figure(figsize=(100,60))
    for i, histogram in enumerate(histograms):
        plt.subplot(48, 3, i+1)
        plt.bar(range(k), histogram)
        plt.xlabel('Codeword')
        plt.ylabel('Frequency')
        plt.title(f'Object Image {i+1}')

    plt.tight_layout()
    plt.show()

    return histograms


In [42]:
x_var=x_variable(obj)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


ValueError: ignored